# GA Data Science 18 (DAT18) - Lab 16

### Recommendation Systems

### Today

1. Simple similarity based recommendation system
2. Recsys

## Similarity based Recommendation System: Beers


Let's build a recommendation system to recommend types of beers based on user reviews

Usual imports (numpy, pandas)

In [3]:
import pandas as pd
import numpy as np

First of all let's get the data

In [4]:
! curl -O https://s3.amazonaws.com/demo-datasets/beer_reviews.tar.gz

curl: (1) Protocol "https" not supported or disabled in libcurl


Import data in a pandas dataframe called "allbeers". Use the compression keyword

In [5]:
# allbeers = pd.read_csv("beer_reviews.tar.gz", compression='gzip')
#If the compression fails - expand the compressed folder and open with:
allbeers = pd.read_csv("beer_reviews/beer_reviews.csv")

Let's look at the data

In [6]:
# allbeers.info()

Let's restrict this to the top 250 beers. Use the value_counts() method select the top 250 beers.
Assign the selected beers to a dataset called df

In [7]:
allbeers.beer_name.value_counts().index

Index([u'90 Minute IPA', u'India Pale Ale',
       u'Old Rasputin Russian Imperial Stout',
       u'Sierra Nevada Celebration Ale', u'Two Hearted Ale',
       u'Stone Ruination IPA', u'Arrogant Bastard Ale',
       u'Sierra Nevada Pale Ale', u'Stone IPA (India Pale Ale)',
       u'Pliny The Elder',
       ...
       u'American Red Cross Ale', u'Green Flash Imperial Red',
       u'Wizard's Wheat', u'Stone Everything But The Kitchen Sink Stout',
       u'Calico Copper E.S.B.', u'Von Jakob American Pilsner',
       u'Magadanskoye Barkhatnoye', u'North Shore Gold',
       u'Ruby River Hefeweizen', u'Two Penny Ale'],
      dtype='object', length=56857)

In [8]:
n = 250
top_n = allbeers.beer_name.value_counts().index[:n]

In [9]:
df = allbeers[allbeers.beer_name.isin(top_n)]
df.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
798,1075,Caldera Brewing Company,1212201268,4.5,4.5,4,grumpy,American Double / Imperial Stout,4.0,4.5,Imperial Stout,NaN,42964
1559,11715,Destiny Brewing Company,1137124057,4.0,3.5,4,blitheringidiot,American Pale Ale (APA),3.5,3.5,Pale Ale,4.5,26420
1560,11715,Destiny Brewing Company,1129504403,4.0,2.5,4,NeroFiddled,American Pale Ale (APA),4.0,3.5,Pale Ale,4.5,26420
1563,11715,Destiny Brewing Company,1137125989,3.5,3.0,4,blitheringidiot,American IPA,4.0,4.0,IPA,NaN,26132
1564,11715,Destiny Brewing Company,1130936611,3.0,3.0,3,Gavage,American IPA,4.0,3.5,IPA,NaN,26132


How big is this dataset?

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 355275 entries, 798 to 1586564
Data columns (total 13 columns):
brewery_id            355275 non-null int64
brewery_name          355275 non-null object
review_time           355275 non-null int64
review_overall        355275 non-null float64
review_aroma          355275 non-null float64
review_appearance     355275 non-null float64
review_profilename    355175 non-null object
beer_style            355275 non-null object
review_palate         355275 non-null float64
review_taste          355275 non-null float64
beer_name             355275 non-null object
beer_abv              353477 non-null float64
beer_beerid           355275 non-null int64
dtypes: float64(6), int64(3), object(4)
memory usage: 37.9+ MB


### Pivot Table

Aggregate the data in a pivot table using the pivot_table method. Display the mean review_overall for each beer_name aggregating the review_overall values by review_profilename. Use the mean as aggregator.

In [11]:
df_pivot = pd.pivot_table(df, values=["review_overall"],
        columns=["beer_name", "review_profilename"],
        aggfunc=np.mean)

In [12]:
#pivot_table converts to a multi-index series. Unstack converts to a dataframe where the last index becomes our column head
df_wide = df_pivot.unstack(-1)
df_wide.head()

review_profilename                         0110x011  02maxima  03SVTCobra  \
               beer_name                                                    
review_overall #9                               NaN       NaN         NaN   
               120 Minute IPA                   NaN       NaN         NaN   
               1554 Enlightened Black Ale       NaN       NaN         NaN   
               60 Minute IPA                    NaN       NaN         NaN   
               90 Minute IPA                      5       NaN         NaN   

review_profilename                         05Harley  0Naught0  0beerguy0  \
               beer_name                                                   
review_overall #9                               NaN       NaN        NaN   
               120 Minute IPA                     4       NaN        NaN   
               1554 Enlightened Black Ale       NaN       NaN        NaN   
               60 Minute IPA                    NaN       NaN        NaN   
               90 Minute IPA                      4       NaN        NaN   

review_profilename                         0runkp0s  0tt0  1000Bottles  \
               beer_name                                                 
review_overall #9                               NaN   NaN          NaN   
               120 Minute IPA                   NaN   1.5          NaN   
               1554 Enlightened Black Ale       NaN   NaN          NaN   
               60 Minute IPA                    NaN   NaN          NaN   
               90 Minute IPA                    NaN   NaN          NaN   

review_profilename                         1001111.0   ...    zuker  \
               beer_name                               ...            
review_overall #9                                NaN   ...      NaN   
               120 Minute IPA                    NaN   ...      NaN   
               1554 Enlightened Black Ale        NaN   ...      NaN   
               60 Minute IPA                     NaN   ...      NaN   
               90 Minute IPA                     NaN   ...      NaN   

review_profilename                         zulufactor  zumicroom  zwalk8  \
               beer_name                                                   
review_overall #9                                 NaN        NaN     NaN   
               120 Minute IPA                     NaN        NaN     NaN   
               1554 Enlightened Black Ale         NaN        NaN     NaN   
               60 Minute IPA                      NaN        NaN     NaN   
               90 Minute IPA                      NaN        NaN     NaN   

review_profilename                         zwoehr  zymrgy  zymurgy4all  \
               beer_name                                                 
review_overall #9                             NaN     NaN          NaN   
               120 Minute IPA                 NaN     NaN          NaN   
               1554 Enlightened Black Ale     NaN     NaN          NaN   
               60 Minute IPA                  NaN     NaN          NaN   
               90 Minute IPA                  NaN     NaN          NaN   

review_profilename                         zymurgywhiz  zythus  zyzygy  
               beer_name                                                
review_overall #9                                  NaN     NaN     NaN  
               120 Minute IPA                      NaN     NaN     NaN  
               1554 Enlightened Black Ale          NaN     NaN     NaN  
               60 Minute IPA                       NaN     NaN     NaN  
               90 Minute IPA                       NaN     NaN     NaN  

[5 rows x 22140 columns]

Display the head of the pivot table, but only for 5 users (columns are users)

In [13]:
df_wide.ix[0:5, 0:5]

review_profilename                         0110x011  02maxima  03SVTCobra  \
               beer_name                                                    
review_overall #9                               NaN       NaN         NaN   
               120 Minute IPA                   NaN       NaN         NaN   
               1554 Enlightened Black Ale       NaN       NaN         NaN   
               60 Minute IPA                    NaN       NaN         NaN   
               90 Minute IPA                      5       NaN         NaN   

review_profilename                         05Harley  0Naught0  
               beer_name                                       
review_overall #9                               NaN       NaN  
               120 Minute IPA                     4       NaN  
               1554 Enlightened Black Ale       NaN       NaN  
               60 Minute IPA                    NaN       NaN  
               90 Minute IPA                      4       NaN

### Discussion: what do you notice in this table?

#### Data munging
Set Nans to zero

In [14]:
df_wide = df_wide.fillna(0)

Check that columns are users

In [15]:
df_wide.columns[:10]

Index([u'0110x011', u'02maxima', u'03SVTCobra', u'05Harley', u'0Naught0',
       u'0beerguy0', u'0runkp0s', u'0tt0', u'1000Bottles', u'1001111.0'],
      dtype='object', name=u'review_profilename')

Check that rows are beers

In [16]:
df_wide.index.levels[0]
beer_names = df_wide.index.levels[1]
beer_names

Index([u'#9', u'120 Minute IPA', u'1554 Enlightened Black Ale',
       u'60 Minute IPA', u'90 Minute IPA',
       u'Aecht Schlenkerla Rauchbier Märzen', u'AleSmith IPA',
       u'AleSmith Speedway Stout', u'Allagash White', u'Alpha King Pale Ale',
       ...
       u'Vanilla Porter', u'Weihenstephaner Hefeweissbier',
       u'Weihenstephaner Korbinian', u'Westmalle Trappist Dubbel',
       u'Westmalle Trappist Tripel', u'World Wide Stout',
       u'Yeti Imperial Stout', u'Young's Double Chocolate Stout',
       u'Yuengling Traditional Lager', u'Éphémère (Apple)'],
      dtype='object', name=u'beer_name', length=250)

In [17]:
df_wide.head(1)

,review_profilename,0110x011,02maxima,03SVTCobra,05Harley,0Naught0,0beerguy0,0runkp0s,0tt0,1000Bottles,1001111.0,...,zuker,zulufactor,zumicroom,zwalk8,zwoehr,zymrgy,zymurgy4all,zymurgywhiz,zythus,zyzygy
,beer_name,,,,,,,,,,,,,,,,,,,,,
review_overall,#9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Calculate distance between beers

We're going to use cosine_similarity from scikit-learn to compute the distance between all beers

Imports

In [18]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import euclidean_distances

Apply cosine similarity to df_wide to calculate pairwise distances

In [19]:
dists = cosine_similarity(df_wide)
dists

array([[ 1.        ,  0.27540494,  0.27410345, ...,  0.32928048,
         0.34805798,  0.31249922],
       [ 0.27540494,  1.        ,  0.25151873, ...,  0.2854835 ,
         0.23301356,  0.2802485 ],
       [ 0.27410345,  0.25151873,  1.        , ...,  0.31629515,
         0.22521858,  0.2737628 ],
       ..., 
       [ 0.32928048,  0.2854835 ,  0.31629515, ...,  1.        ,
         0.28025764,  0.34504013],
       [ 0.34805798,  0.23301356,  0.22521858, ...,  0.28025764,
         1.        ,  0.25526913],
       [ 0.31249922,  0.2802485 ,  0.2737628 , ...,  0.34504013,
         0.25526913,  1.        ]])

### Discussion: what type of object is dists?

Convert dists to a Pandas DataFrame, use the index as column index as well (distances are a square matrix).

In [20]:
dists = pd.DataFrame(dists)
dists.columns = beer_names
dists.index = beer_names
dists.ix[0:10, 0:10]

beer_name,#9,120 Minute IPA,1554 Enlightened Black Ale,60 Minute IPA,90 Minute IPA,Aecht Schlenkerla Rauchbier Märzen,AleSmith IPA,AleSmith Speedway Stout,Allagash White,Alpha King Pale Ale
beer_name,,,,,,,,,,
#9,1.000000,0.275405,0.274103,0.388364,0.365175,0.253841,0.228479,0.227612,0.340681,0.293315
120 Minute IPA,0.275405,1.000000,0.251519,0.378258,0.410366,0.262425,0.315971,0.337541,0.282273,0.336796
1554 Enlightened Black Ale,0.274103,0.251519,1.000000,0.319887,0.314028,0.252486,0.266866,0.261761,0.260275,0.307296
60 Minute IPA,0.388364,0.378258,0.319887,1.000000,0.533042,0.316928,0.312343,0.307627,0.360975,0.385249
90 Minute IPA,0.365175,0.410366,0.314028,0.533042,1.000000,0.312861,0.344218,0.358754,0.356804,0.418582
Aecht Schlenkerla Rauchbier Märzen,0.253841,0.262425,0.252486,0.316928,0.312861,1.000000,0.244490,0.246063,0.297672,0.263248
AleSmith IPA,0.228479,0.315971,0.266866,0.312343,0.344218,0.244490,1.000000,0.521889,0.277409,0.400741
AleSmith Speedway Stout,0.227612,0.337541,0.261761,0.307627,0.358754,0.246063,0.521889,1.000000,0.273930,0.420247
Allagash White,0.340681,0.282273,0.260275,0.360975,0.356804,0.297672,0.277409,0.273930,1.000000,0.295666


Select some beers and look their distances to other beers

In [21]:
beers_i_like = ['Sierra Nevada Pale Ale', '120 Minute IPA', 'Allagash White']
dists[beers_i_like].head()

beer_name,Sierra Nevada Pale Ale,120 Minute IPA,Allagash White
beer_name,,,
#9,0.373968,0.275405,0.340681
120 Minute IPA,0.301693,1.000000,0.282273
1554 Enlightened Black Ale,0.330033,0.251519,0.260275
60 Minute IPA,0.459641,0.378258,0.360975
90 Minute IPA,0.441189,0.410366,0.356804


Sum the distances of my favourite beers by row, to have one distance from each beer in the sample

In [22]:
beers_summed = dists[beers_i_like].apply(lambda row: np.sum(row), axis=1)
# beers_summed = np.sum(dists[beers_i_like], axis=1)

#### Performance

Optional: which one is faster? use ```%timeit``` to check

In [23]:
%timeit dists[beers_i_like].apply(lambda row: np.sum(row), axis=1)

100 loops, best of 3: 6.07 ms per loop


In [24]:
%timeit np.sum(dists[beers_i_like], axis=1)

1000 loops, best of 3: 474 µs per loop


#### Ranking

Sort summed beers from best to worse

In [25]:
beers_summed = beers_summed.order(ascending=False)
beers_summed

C:\Users\cVen\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: order is deprecated, use sort_values(...)
  if __name__ == '__main__':


beer_name
Sierra Nevada Pale Ale                        1.654205
Allagash White                                1.634784
120 Minute IPA                                1.583966
HopDevil Ale                                  1.224217
Sierra Nevada Celebration Ale                 1.215156
90 Minute IPA                                 1.208359
60 Minute IPA                                 1.198874
Stone Ruination IPA                           1.194210
Stone IPA (India Pale Ale)                    1.193193
Storm King Stout                              1.192405
Arrogant Bastard Ale                          1.189981
Sierra Nevada Bigfoot Barleywine Style Ale    1.178245
Prima Pils                                    1.178093
Brooklyn Black Chocolate Stout                1.156365
Ayinger Celebrator Doppelbock                 1.148356
Hennepin (Farmhouse Saison)                   1.147501
Samuel Adams Boston Lager                     1.146304
Hop Rod Rye                                   1.140271


Filter out the beers used as input and transform to list

In [26]:
ranked_beers = beers_summed.index[~beers_summed.index.isin(beers_i_like)]
ranked_beers = ranked_beers.tolist()
ranked_beers[:10]

['HopDevil Ale',
 'Sierra Nevada Celebration Ale',
 '90 Minute IPA',
 '60 Minute IPA',
 'Stone Ruination IPA',
 'Stone IPA (India Pale Ale)',
 'Storm King Stout',
 'Arrogant Bastard Ale',
 'Sierra Nevada Bigfoot Barleywine Style Ale',
 'Prima Pils']

### Pair Programming!

Define a function that does what we just did for an arbitrary input list of beers. it should also receive the maximum number of beers requested n as optional parameter.

In [27]:
def get_similar(beers, n=None):
    """
    calculates which beers are most similar to the beers provided. Does not return
    the beers that were provided
    
    Parameters
    ----------
    beers: list
        some beers!
    
    Returns
    -------
    ranked_beers: list
        rank ordered beers
    """
    
#     beers2 = []
#     for beer in beers:
#         if beer in dists.columns
#             beers2.append(beer)
    
    beers = [beer for beer in beers if beer in dists.columns]
    beers_summed = dists[beers].apply(lambda row: np.sum(row), axis=1)
    beers_summed = beers_summed.order(ascending=False)
    ranked_beers = beers_summed.index[beers_summed.index.isin(beers)==False]
    ranked_beers = ranked_beers.tolist()
    if n is None:
        return ranked_beers
    else:
        return ranked_beers[:n]

Test your function. Find the 10 beers most similar to "120 Minute IPA"

In [28]:
get_similar(["120 Minute IPA"], 10)

C:\Users\cVen\Anaconda2\lib\site-packages\ipykernel\__main__.py:24: FutureWarning: order is deprecated, use sort_values(...)


['World Wide Stout',
 '90 Minute IPA',
 'Double Bastard Ale',
 'Stone Ruination IPA',
 'Stone Imperial Russian Stout',
 'Storm King Stout',
 '60 Minute IPA',
 'Oaked Arrogant Bastard Ale',
 'Sierra Nevada Bigfoot Barleywine Style Ale',
 'Brooklyn Black Chocolate Stout']

In [29]:
for beer in get_similar(["120 Minute IPA"], 10):
    print beer

World Wide Stout
90 Minute IPA
Double Bastard Ale
Stone Ruination IPA
Stone Imperial Russian Stout
Storm King Stout
60 Minute IPA
Oaked Arrogant Bastard Ale
Sierra Nevada Bigfoot Barleywine Style Ale
Brooklyn Black Chocolate Stout


C:\Users\cVen\Anaconda2\lib\site-packages\ipykernel\__main__.py:24: FutureWarning: order is deprecated, use sort_values(...)


Try again with the 10 beers most similar to ["Coors Light", "Bud Light", "Amstel Light"]

In [30]:
for i, beer in enumerate(get_similar(["Coors Light", "Bud Light", "Amstel Light"], 10)):
    print "%d) %s" % (i+1, beer)

1) Miller Lite
2) Budweiser
3) Corona Extra
4) Samuel Adams Boston Lager
5) Heineken Lager Beer
6) Blue Moon Belgian White
7) Guinness Draught
8) Miller High Life
9) Samuel Adams Summer Ale
10) Sierra Nevada Pale Ale


C:\Users\cVen\Anaconda2\lib\site-packages\ipykernel\__main__.py:24: FutureWarning: order is deprecated, use sort_values(...)


Optional: register an account on yhat and deploy your model following the instructions [here](https://docs.yhathq.com/python/examples/beer-recommender) and [here](http://nbviewer.ipython.org/gist/glamp/20a18d52c539b87de2af)

## Recsys

A python library for implementing a recommender system. If you'd like to, I recommend you explore this project. It's an efficient way to get a recommendation engine off the ground. The example below uses SVD.

In [31]:
"""
##install python-recsys

### first install dependencies

pip install csc-pysparse networkx divisi2

### then install recsys
git clone https://github.com/python-recsys/python-recsys.git
cd python-recsys/

python setup.py install
"""

'\n##install python-recsys\n\n### first install dependencies\n\npip install csc-pysparse networkx divisi2\n\n### then install recsys\ngit clone https://github.com/python-recsys/python-recsys.git\ncd python-recsys/\n\npython setup.py install\n'

In [32]:
! pip install csc-pysparse networkx divisi2

  Using cached csc-pysparse-1.1.1.4.tar.gz
  Using cached Divisi2-2.2.5.tar.gz
  Running setup.py bdist_wheel for csc-pysparse
  Complete output from command C:\Users\cVen\Anaconda2\python.exe -c "import setuptools;__file__='c:\\users\\cven\\appdata\\local\\temp\\pip-build-a4mk_q\\csc-pysparse\\setup.py';exec(compile(open(__file__).read().replace('\r\n', '\n'), __file__, 'exec'))" bdist_wheel -d c:\users\cven\appdata\local\temp\tmpacvpkopip-wheel-:
  blas_opt_info:
  blas_mkl_info:
    libraries mkl_lapack95_lp64,mkl_blas95_lp64,mkl_intel_lp64,mkl_intel_thread,mkl_core,libiomp5md,libifportmd not found in []
    NOT AVAILABLE
  
  openblas_info:
    libraries openblas not found in ['C:\\Users\\cVen\\Anaconda2\\lib', 'C:\\', 'C:\\Users\\cVen\\Anaconda2\\libs']
    NOT AVAILABLE
  
  atlas_3_10_blas_threads_info:
  Setting PTATLAS=ATLAS
    libraries tatlas not found in ['C:\\Users\\cVen\\Anaconda2\\lib', 'C:\\', 'C:\\Users\\cVen\\Anaconda2\\libs']
    NOT AVAILABLE
  
  atlas_3_10_blas_i

  Failed building wheel for csc-pysparse
  Failed building wheel for divisi2
Command "C:\Users\cVen\Anaconda2\python.exe -c "import setuptools, tokenize;__file__='c:\\users\\cven\\appdata\\local\\temp\\pip-build-a4mk_q\\csc-pysparse\\setup.py';exec(compile(getattr(tokenize, 'open', open)(__file__).read().replace('\r\n', '\n'), __file__, 'exec'))" install --record c:\users\cven\appdata\local\temp\pip-xdkstu-record\install-record.txt --single-version-externally-managed --compile" failed with error code 1 in c:\users\cven\appdata\local\temp\pip-build-a4mk_q\csc-pysparse


Load recsys.algotihm, set VERBOSE = True import SVD class

In [33]:
import recsys.algorithm
recsys.algorithm.VERBOSE = True
from recsys.algorithm.factorize import SVD

ImportError: No module named csc

Let's look at the files

In [ ]:
! ls ../data/movielens/

Import 'movies.dat' to a 'movies' pandas dataframe. Make sure you name the columns, use the correct separator and define the index.

In [ ]:
movies = pd.read_table('../data/movielens/movies.dat', sep='::', engine='python',
                       names= ['ITEMID', 'Title', 'Genres'], index_col= 'ITEMID')

In [ ]:
movies.head()

Import 'ratings.dat' to a 'ratings' pandas dataframe. Make sure you name the columns, use the correct separator.

In [ ]:
ratings = pd.read_table('../data/movielens/ratings.dat', sep='::', engine='python',
                        names= ['UserID','MovieID','Rating','Timestamp'])

In [ ]:
ratings.head()

Initialize an SVD instance

In [ ]:
svd = SVD()

Populate it with the data from the ratings dataset, using the built in load_data method

In [ ]:
svd.load_data(filename='../data/movielens/ratings.dat', sep='::', format={'col':0, 'row':1, 'value':2, 'ids': int})

Compute SVD

$M=U \Sigma V^T$:

In [ ]:
k = 100
svd.compute(k=k, min_values=10, pre_normalize=None, mean_center=True, post_normalize=True)

you can also save the output SVD model (in a zip file)

In [ ]:
# svd.compute(k=k, min_values=10, pre_normalize=None, mean_center=True, post_normalize=True, savefile='/tmp/movielens')

Reload a saved model:

In [ ]:
# svd2 = SVD(filename='/tmp/movielens')

Find the ITEMID number for "Toy Story (1995)"

In [ ]:
movies[movies.Title == "Toy Story (1995)"]

Find the ITEMID number for "Bug's Life, A (1998)"

In [ ]:
movies[movies.Title == "Bug's Life, A (1998)"]

Compute similarity between the two movies

In [ ]:
ITEMID1 = 1    # Toy Story (1995)
ITEMID2 = 2355 # A bug's life (1998)
print svd.similarity(ITEMID1, ITEMID2)
print svd2.similarity(ITEMID1, ITEMID2)

Get movies similar to Toy Story

In [ ]:
svd.similar(ITEMID1)

In [ ]:
movies[movies.index == 3114]

Predict rating for a given user and movie, $\hat{r}_{ui}$

In [ ]:
# ratings[(ratings.UserID == 1) & (ratings.Rating == 5)]
# ratings.columns

In [ ]:
MIN_RATING = 0.0
MAX_RATING = 5.0
ITEMID = 1
USERID = 1
svd.predict(ITEMID, USERID, MIN_RATING, MAX_RATING)

In [ ]:
svd.get_matrix().value(ITEMID, USERID)

Recommend non rated movies to a user

In [ ]:
svd.recommend(USERID, is_row=False)

Which users should see Toy Story? (e.g. which users -that have not rated Toy Story- would give it a high rating?)

In [ ]:
svd.recommend(ITEMID)

Find out more here: [https://github.com/ocelma/python-recsys](https://github.com/ocelma/python-recsys)